In [ ]:
#!pip install keybert spacy sklearn sentence_transformers langchain

# Продвинутые техники RAG

## 1. Сжатие промптов

### 1.1. Зачем нужно сжатие промптов

**Проблема ресурсоемкости**: Каждый запрос в LLM (Language Model) ограничен по количеству символов, а большие промпты требуют значительных вычислительных мощностей. Сокращение промпта — это способ повысить производительность и сделать процесс более экономичным.

**Улучшение скорости обработки**: Сжатие промптов ускоряет время обработки запроса и может повысить общую точность ответа, так как модель обрабатывает только ключевую информацию.

**Фокус на релевантной информации**: В больших промптах часто много избыточной информации, и задача сжатия — выделить только необходимые элементы.

### 1.2. Методы сжатия

##### Метод 1: Сжатие на основе ключевых слов (Keyword Extraction)  

Ключевые слова помогают выделить основные аспекты запроса, которые будут важны для генерации ответа. Один из популярных инструментов для этого — библиотека KeyBERT, которая использует BERT для нахождения ключевых слов в тексте.


**Преимущества**: Простота и быстрая реализация. Полезно в ситуациях, когда запрос достаточно длинный и содержит много уточняющих деталей.  

**Недостатки**: Иногда ключевые слова могут не учитывать контекст, и часть важной информации может быть утрачена.

In [ ]:
from keybert import KeyBERT

# Инициализация модели KeyBERT
kw_model = KeyBERT()

def compress_prompt_keywords(query):
    # Извлечение топ-5 ключевых слов
    keywords = kw_model.extract_keywords(query, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=5)
    return " ".join([word[0] for word in keywords])

# Пример использования
query = "What are the main advantages of using advanced RAG techniques for improving query responses?"
compressed_query = compress_prompt_keywords(query)
print("Compressed Query (Keywords):", compressed_query)

Compressed Query (Keywords): rag techniques query responses advanced rag advantages using rag


##### Метод 2: Сжатие на основе извлечения сущностей (Named Entity Recognition, NER)

С помощью NER можно выделить важные сущности, такие как имена людей, даты, локации, которые несут смысловую нагрузку в запросе. Этот метод эффективен для извлечения основных фактов из длинных запросов.

**Преимущества**: Высокая точность при работе с фактологической информацией.  

**Недостатки**: При запросах, содержащих описания или абстрактные темы, данный метод может терять важные детали.

In [ ]:
import spacy

!python -m spacy download en_core_web_sm

# Загрузка предобученной модели spaCy
nlp = spacy.load("en_core_web_sm")

def compress_prompt_entities(query):
    doc = nlp(query)
    # Извлечение только нужных типов сущностей
    entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "DATE", "TIME"]]
    return " ".join(entities)

# Пример использования
query = "Describe the impact of Corrective RAG introduced by OpenAI in 2024 on information retrieval."
compressed_query = compress_prompt_entities(query)
print("Compressed Query (Entities):", compressed_query)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 8.3 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB 10.5 MB/s eta 0:00:01
     --------------- ------------------------ 5.0/12.8 MB 10.1 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 10.2 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 10.8 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Compressed Query (Entities): Corrective RAG OpenAI 2024


##### Метод 3: Сжатие с использованием TF-IDF (Term Frequency - Inverse Document Frequency)

TF-IDF помогает выбрать термины, которые наиболее важны для конкретного запроса, на основе их частоты. Этот метод учитывает не только популярные слова, но и те, которые важны именно для данного запроса.

**Преимущества**: Подходит для запросов с большим объемом текста, так как учитывает уникальность терминов.  

**Недостатки**: Требует дополнительных данных (контекста) для лучшего определения важных терминов.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compress_prompt_tfidf(query, documents):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([query] + documents)
    feature_array = vectorizer.get_feature_names_out()
    tfidf_sorting = tfidf_matrix[0].toarray().flatten().argsort()[::-1]
    # Извлечение топ-5 терминов
    top_n = tfidf_sorting[:5]
    return " ".join([feature_array[i] for i in top_n])

# Пример использования
documents = ["RAG is a technique for using external data in generation.", "Corrective RAG improves response accuracy."]
query = "What are the main advancements in RAG models introduced by OpenAI?"
compressed_query = compress_prompt_tfidf(query, documents)
print("Compressed Query (TF-IDF):", compressed_query)

Compressed Query (TF-IDF): what introduced the advancements are


##### Метод 4: Семантическое сжатие с помощью Sentence-BERT

Модель Sentence-BERT создает компактные векторные представления для запросов и текстов, позволяя сохранять семантическое сходство. Это полезно для сжатия запроса в один или два важных предложения, сохраняя его значение.

**Преимущества**: Сохраняет высокий уровень семантического сходства между исходным запросом и сжатым текстом.  

**Недостатки**: Зависит от качества обучающих данных и параметров модели, требует дополнительных вычислительных ресурсов.

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Инициализация модели Sentence-BERT
model = SentenceTransformer('all-MiniLM-L6-v2')

def compress_prompt_bert(query, documents):
    query_embedding = model.encode(query, convert_to_tensor=True)
    doc_embeddings = model.encode(documents, convert_to_tensor=True)

    # Нахождение наиболее релевантного документа
    scores = util.pytorch_cos_sim(query_embedding, doc_embeddings).squeeze()
    top_idx = scores.argmax().item()

    # Проверка порогового значения
    if scores[top_idx] > 0.5:
        return documents[top_idx]
    else:
        return query

# Пример использования
documents = ["Advanced RAG models use corrective techniques.", "Adaptive RAG allows for flexible query adjustments."]
query = "Explain the latest advancements in Retrieval-Augmented Generation."
compressed_query = compress_prompt_bert(query, documents)
print("Compressed Query (BERT):", compressed_query)

Compressed Query (BERT): Explain the latest advancements in Retrieval-Augmented Generation.


### 1.3. Оптимизация запросов

**Адаптация к сложности задачи**: Оптимизация позволяет выбрать параметры модели в зависимости от сложности запроса. Например, для длинных и сложных запросов может быть полезно задать более низкий параметр temperature, чтобы получить более последовательный и конкретный ответ.  

**Управление длиной ответа**: Настройка параметра max_tokens для ограничения длины генерируемого ответа.  

**Контроль уровня детальности**: Например, использование top_p и frequency_penalty, чтобы избежать избыточной детализации или чрезмерных повторений.

In [ ]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('.env')

True

In [ ]:
from langchain.llms import OpenAI

def optimized_query(query):
    # Настройка параметров на основе длины запроса
    if len(query) > 50:
        llm = OpenAI(temperature=0.2, max_tokens=150, top_p=0.85)
    else:
        llm = OpenAI(temperature=0.7, max_tokens=100, top_p=0.95)

    # Выполнение запроса
    response = llm(query)
    return response

# Пример использования
query = "What are the primary benefits of using Adaptive RAG in large-scale information systems?"
response = optimized_query(query)
print("Optimized Query Response:", response)

C:\Users\basil\AppData\Local\Temp\ipykernel_11316\580912320.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(query)


Optimized Query Response: 

1. Real-time monitoring and reporting: Adaptive RAG allows for real-time monitoring and reporting of data in large-scale information systems. This enables organizations to quickly identify and address any issues or anomalies, leading to improved decision-making and problem-solving.

2. Flexibility and adaptability: Adaptive RAG can be customized and adapted to the specific needs and requirements of different information systems. This flexibility allows for better integration with existing systems and processes, making it easier to manage and analyze large amounts of data.

3. Improved data accuracy: By continuously monitoring and updating data, Adaptive RAG helps to ensure the accuracy and reliability of information in large-scale systems. This reduces the risk of errors and improves the overall quality of data.

4. Enhanced data visualization:


**Temperature**: Контролирует степень вариативности в ответе. Более высокие значения дают более креативные ответы.  

**Max Tokens**: Ограничивает длину ответа, что полезно для упрощения обработки.  

**Top-p**: Применяет фильтрацию по вероятности, позволяя модели генерировать более целенаправленные ответы.

## 2. Продвинутые техники RAG

### 2.1. Введение в продвинутые техники RAG

#### 2.1.1. Проблемы базового RAG или почему стандартного RAG недостаточно?

Retrieval-Augmented Generation (RAG) — это подход, который объединяет поиск информации и генерацию текста, что позволяет LLM (Large Language Models) получать доступ к информации из внешних источников, таких как базы данных, Википедия и другие документы. Стандартный RAG состоит из двух основных этапов:

- **Retrieval (Поиск):** выбор наиболее релевантных документов для запроса.
- **Generation (Генерация):** использование этих документов для генерации ответа с помощью модели.

Хотя RAG значительно улучшает способность моделей к обработке информации, у этого метода есть несколько ограничений:

1. **Зависимость от качества и релевантности извлечённых данных:** стандартный RAG полагается на несколько документов, которые могут не всегда идеально соответствовать запросу. Если подобранные документы недостаточно точные, это напрямую влияет на качество ответа.
2. **Ограниченная способность к корректировке неверных данных:** модель RAG генерирует текст на основе извлечённой информации, но при наличии несоответствий или неверной информации в документах, модель не может автоматически исправить эти ошибки.
3. **Отсутствие адаптивности к сложности запроса:** стандартный RAG не учитывает специфику запроса. Он одинаково обрабатывает простые и сложные запросы, что может приводить к избыточной или недостаточной информации в ответе.
4. **Ресурсоемкость и скорость:** стандартный RAG требует значительных вычислительных ресурсов для работы с большими наборами данных. Извлечение и генерация могут быть медленными и дорогими при обработке сложных или масштабных запросов.

Эти ограничения часто становятся особенно заметными в приложениях, где точность и надежность информации играют критическую роль — например, в медицинских системах, юридических консультациях и научных исследованиях.



#### 2.1.2. Какие задачи требуют продвинутых техник RAG?

Продвинутые техники RAG разработаны для того, чтобы решать задачи, которые стандартный RAG не может выполнить на должном уровне. Рассмотрим несколько сценариев, где использование стандартного RAG приводит к неудовлетворительным результатам и требуется внедрение улучшенных методов:

1. **Глубокий анализ и корректировка ответов:** В некоторых случаях пользователи хотят не только получить ответ, но и убедиться, что ответ точно соответствует запросу. Если стандартный RAG даёт недостаточно точный результат, требуется механизм корректировки для повышения качества.
   
2. **Адаптация модели к уровню сложности запроса:** в приложениях, где запросы имеют сильно различающиеся уровни сложности, необходимы методы, которые могут "подстраиваться" под каждый конкретный запрос, обеспечивая максимальную релевантность и точность ответа.

3. **Устранение противоречий в данных:** данные, извлечённые из разных источников, могут содержать противоречивую информацию. Стандартный RAG не предназначен для устранения таких расхождений, и продвинутые методы, такие как Corrective RAG, могут помочь в решении этой проблемы.

4. **Контекстно-зависимые запросы:** в некоторых системах требуется обработка запросов с учётом истории взаимодействий пользователя или предыдущих запросов. Стандартный RAG не сохраняет "память" об этих взаимодействиях, поэтому необходимо использовать подходы, позволяющие учитывать прошлую информацию, как в Self-RAG.

5. **Универсальность в обработке разных типов данных:** стандартный RAG ограничен текстовыми источниками. Если запросы касаются данных разного формата (например, графов, изображений, аудиофайлов), нужны методы, способные адаптироваться к этим особенностям и работать с разными типами данных.



#### 2.1.3. Основные направления улучшений и продвинутые техники RAG

Существуют несколько подходов к улучшению RAG, каждый из которых решает определённый набор задач. Рассмотрим основные техники и их цели:

#### 1. Corrective RAG

- **Назначение:** направлен на корректировку ответов, полученных от RAG, для улучшения их точности.
- **Принцип работы:** включает дополнительный модуль, который анализирует ответ, сгенерированный моделью, и проверяет его на соответствие исходному запросу. Если ответ содержит неточности, он корректируется, и генерируется новый, уточнённый вариант.
- **Применение:** подходит для областей, где важна высокая точность, таких как медицина или право, где ошибка в ответе может привести к негативным последствиям.

#### 2. Self-RAG

- **Назначение:** Self-RAG позволяет системе сохранять "память" о прошлых запросах и ответах, обеспечивая более контекстно-зависимую генерацию ответов.
- **Принцип работы:** вместо простого поиска релевантных документов Self-RAG также обращает внимание на предыдущие запросы и ответы, помогая таким образом сохранить связность и непрерывность взаимодействия с пользователем.
- **Применение:** идеален для сервисов поддержки клиентов или консультационных систем, где важен учет контекста предыдущих вопросов.

#### 3. Adaptive RAG

- **Назначение:** автоматическая адаптация параметров генерации ответа в зависимости от сложности и длины запроса.
- **Принцип работы:** в Adaptive RAG параметры модели, такие как `temperature`, `max_tokens`, и другие, изменяются динамически в зависимости от структуры запроса. Например, более длинные запросы могут обрабатываться с меньшей температурой для увеличения согласованности ответа, тогда как более простые запросы могут требовать большей вариативности.
- **Применение:** эффективен для приложений, где вопросы могут варьироваться от очень простых до сложных, требующих развернутых ответов.

#### 4. Prompt Compression and Query Optimization

- **Назначение:** упрощение и оптимизация входных данных (запросов), чтобы они содержали только ключевую информацию и занимали меньше ресурсов.
- **Принцип работы:** включает предварительную обработку запросов для извлечения наиболее значимых элементов текста, таких как ключевые слова, сущности или важные фразы, что помогает снизить нагрузку на систему.
- **Применение:** полезен в ситуациях, где важно экономить вычислительные ресурсы или обрабатывать большие объемы запросов в ограниченные сроки.


### 2.2. Corrective RAG

[Shi-Qi Yan and Jia-Chen Gu and Yun Zhu and Zhen-Hua Ling. Corrective Retrieval Augmented Generation](https://arxiv.org/abs/2401.15884)

#### 2.2.1. Зачем нужен Corrective RAG?

Corrective RAG необходим в ситуациях, когда стандартный RAG допускает ошибки при генерации ответа. Эти ошибки могут возникать по нескольким причинам:

1. **Неоднозначность запроса:**
   - некоторые запросы могут включать сложные или двусмысленные формулировки. Стандартный RAG не всегда способен корректно интерпретировать такие запросы, что приводит к ответам, не полностью отражающим суть вопроса.
   
2. **Противоречивость данных:**
   - при извлечении информации из нескольких источников некоторые данные могут конфликтовать между собой. Например, один документ может содержать одну интерпретацию событий, а другой – противоположную. В таких случаях модель может не выбрать наиболее подходящий источник, что приведет к некорректной генерации.

3. **Частичная релевантность:**
   - документы, выбранные стандартным RAG, могут частично подходить к запросу, но не полностью, и в результате ответ оказывается неполным или неточным.

4. **Роль пользователя в критически важных системах:**
   - В ряде сфер (например, здравоохранение, финансы или право) ошибки в ответах могут иметь серьезные последствия. Там, где требуется максимальная точность, Corrective RAG может помочь минимизировать риски.


#### 2.2.2. Принцип работы Corrective RAG

Corrective RAG добавляет ещё один слой контроля и корректировки после этапа генерации. Основные компоненты архитектуры Corrective RAG:

1. **Модуль извлечения информации (Retrieval):**
   - на этом этапе модель извлекает релевантные документы, как и в стандартном RAG, но при этом используется улучшенная фильтрация и предобработка данных для минимизации рисков.
   
2. **Генерация (Generation):**
   - модель формирует начальный ответ на основе извлечённых данных, однако этот ответ ещё не считается окончательным.
   
3. **Модуль коррекции (Corrective Module):**
   - на этом этапе система проверяет сгенерированный ответ на соответствие запросу и наличию ошибок. Этот модуль может использовать дополнительные модели и алгоритмы для анализа текста и выявления противоречий.

4. **Повторная генерация или уточнение ответа:**
   - если модуль коррекции обнаруживает ошибки или неточности, система может повторить генерацию с уточнёнными параметрами, чтобы добиться лучшего результата.

#### 2.2.3. Методы корректировки в Corrective RAG

Существует несколько подходов к корректировке ошибок, которые применяются в Corrective RAG:

1. **Сравнение с ключевыми фактами и сущностями:**
   - Этот метод использует заранее определенные сущности и ключевые факты, которые модель должна включить в ответ. Модуль коррекции проверяет, упоминаются ли эти элементы в ответе. Если какой-то важный факт отсутствует, модель может перегенерировать ответ, чтобы включить его.
   
2. **Анализ противоречий:**
   - Система может использовать алгоритмы на основе правил или модели, чтобы проверить, не содержит ли ответ противоречий. Например, если ответ ссылается на разные даты для одного и того же события, модель корректирует это расхождение.

3. **Контекстная проверка запроса и ответа:**
   - Модуль коррекции может оценивать, насколько ответ подходит к контексту запроса. Если ответ кажется слишком общим или не полностью отвечает на вопрос, модель уточняет ответ, добавляя нужные детали.

4. **Коррекция с помощью дополнительных примеров:**
   - Corrective RAG может использовать базу дополнительных примеров ответов, чтобы сопоставить сгенерированный ответ с эталонными ответами и скорректировать его на основе найденных отклонений.


#### 2.2.4. Пример реализации Corrective RAG

Рассмотрим реализацию Corrective RAG с использованием реального ретривера на основе TF-IDF в LangChain. В данном примере создается база данных документов о достижениях Никола Теслы и применяется коррекция ответа для уточнения результата.

In [ ]:
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import TFIDFRetriever
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import Document

# База документов для примера, преобразуем каждый документ в объект Document
documents = [
    Document(page_content="Никола Тесла был инженером и изобретателем, который разработал систему переменного тока."),
    Document(page_content="Его вклад в электротехнику включает создание индукционного мотора и генератора переменного тока."),
    Document(page_content="Тесла также работал над передовыми технологиями передачи энергии и радиосвязи."),
    Document(page_content="Известно, что Тесла разработал первый индукционный двигатель, работающий на переменном токе."),
    Document(page_content="Никола Тесла является одним из наиболее значительных учёных в области электротехники и физики."),
]

# Разделение документов на фрагменты для улучшения поиска
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

# Создание индекса TF-IDF для поиска по текстам
# Создаем новый список документов для TFIDFRetriever
retriever = TFIDFRetriever.from_documents(texts)

# Настройка модели и QA-цепочки для генерации ответа
llm = OpenAI(temperature=0.2)
qa_chain = load_qa_chain(llm=llm, chain_type="stuff")

# Определяем запрос и запускаем генерацию
query = "Расскажите о достижениях Никола Тесла в области электричества."
print("Исходный запрос:", query)
relevant_texts = retriever.get_relevant_documents(query)
initial_answer = qa_chain.run(input_documents=relevant_texts, question=query)

# Определяем модуль коррекции
def corrective_module(answer, query):
    # Определяем ключевые сущности для данного запроса
    required_entities = ["Никола Тесла", "электричество", "достижения", "генератор переменного тока"]

    # Проверяем, содержит ли ответ все необходимые сущности
    missing_entities = [entity for entity in required_entities if entity not in answer]

    if missing_entities:
        # Если не хватает сущностей, корректируем запрос
        refined_query = f"{query}. Пожалуйста, укажите: {', '.join(missing_entities)}."
        print("Исправленный запрос:", refined_query)  # Печать исправленного запроса
        # Повторный поиск и генерация ответа
        relevant_texts = retriever.get_relevant_documents(refined_query)
        corrected_answer = qa_chain.run(input_documents=relevant_texts, question=refined_query)
        return corrected_answer
    return answer

# Применение коррекции
final_answer = corrective_module(initial_answer, query)
print("Ответ после корректировки:", final_answer)

Исходный запрос: Расскажите о достижениях Никола Тесла в области электричества.
Исправленный запрос: Расскажите о достижениях Никола Тесла в области электричества.. Пожалуйста, укажите: электричество, генератор переменного тока.
Ответ после корректировки:  Никола Тесла был известен своими значительными достижениями в области электричества. Он разработал систему переменного тока, которая стала основой для современных систем электроснабжения. Тесла также изобрел индукционный мотор и генератор переменного тока, которые существенно улучшили производительность и эффективность электрических систем. Он также работал над передовыми технологиями передачи энергии и радиосвязи, что сделало его одним из наиболее значимых ученых в области электротехники и физики.


**Пояснение**:

1. **База документов**:
   - добавляем документы о достижениях Теслы в электротехнике.
2. **Text Splitter**:
   - этот компонент разбивает большие документы на небольшие фрагменты, чтобы улучшить поиск и сделать извлечение информации более точным.
3. **TF-IDF индекс**:
   - используется для эффективного извлечения наиболее релевантных документов из базы данных на основе текста запроса.


**Пример работы кода**:
1. **Первичный запрос**: "Расскажите о достижениях Никола Тесла в области электричества."
2. **Первоначальный ответ**: "Никола Тсела был инженером и изобретателем, который внёс большой вклад в электротехнику."
3. **Корректировка**: Модуль коррекции добавляет уточнение, и итоговый запрос становится: "Расскажите о достижениях Никола Тесла в области электричества. Пожалуйста, укажите: генератор переменного тока."
4. **Корректированный ответ**: "Никола Тесла разработал генератор переменного тока, который сыграл ключевую роль в развитии современной электротехники."

#### Заключение

Corrective RAG представляет собой улучшенную версию стандартного RAG, которая включает механизм обратной связи для корректировки ошибок модели при генерации ответов. Этот подход особенно эффективен в случаях, когда:
- **Запросы требуют высокой точности:** Corrective RAG позволяет снизить количество ошибок и неполных ответов, применяя обратную связь и донастройку ретривера для повышения качества извлеченных данных.
- **Процесс обучения модели недостаточен:** В ситуациях, когда стандартная модель не справляется с задачей из-за отсутствия специфических знаний, Corrective RAG помогает корректировать и дорабатывать ответы, используя информацию, полученную в ходе предыдущих взаимодействий.
- **Сложные вопросы требуют итеративного подхода:** Corrective RAG поддерживает цикличную доработку ответа, что особенно полезно для многослойных вопросов, требующих уточнений и доработок.

Corrective RAG идеально подходит для приложений, где важна точность и требуется подробный анализ, таких как юридические консультации, техническая поддержка или обучение. За счет обратной связи и корректировки извлекаемых данных данный метод помогает избежать распространенных ошибок и повышает удовлетворенность пользователей.

Реализация Corrective RAG с использованием LangGraph [Corrective RAG (CRAG) using LangGraph](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag/)

### 2.3. Self-RAG

#### 2.3.1. Введение в Self-RAG

**Self-RAG (Self-Retrieving Augmented Generation)** — это подход, при котором модель сама инициирует дополнительные запросы к системе поиска, чтобы обогатить свой ответ. Self-RAG полезен в случаях, когда исходный ответ неполон, а модель должна сама решить, какие дополнительные сведения нужны для завершения ответа.

**Особенности Self-RAG:**
- **Самостоятельное уточнение запроса:** Self-RAG позволяет модели анализировать ответ и определять, когда и какие дополнительные данные могут быть полезны.
- **Обработка сложных запросов:** Self-RAG применяется для сложных вопросов, которые требуют нескольких этапов поиска и генерации информации.
- **Гибкость и адаптивность:** Использование Self-RAG делает модель более гибкой, поскольку она способна решать, что требуется для создания более полного ответа.


#### 2.3.2. Проблемы, решаемые Self-RAG

1. **Неполнота информации в одном запросе:**
   - иногда начальный запрос не может охватить всю информацию, необходимую для создания полноценного ответа. Self-RAG помогает преодолеть это, выполняя дополнительные запросы по мере необходимости.

2. **Автоматизация поиска дополнительных данных:**
   - без Self-RAG дополнительные запросы и уточнения обычно задаются вручную. В Self-RAG этот процесс автоматизируется, что упрощает получение необходимых данных.

3. **Минимизация ручного вмешательства:**
   - self-RAG позволяет модели быть автономной в извлечении информации. Это снижает необходимость в ручном вмешательстве и делает процесс более эффективным.


#### 2.3.3. Применение Self-RAG на практике

Примером может служить ситуация, когда система отвечает на сложные исторические или научные запросы, которые требуют нескольких уточнений для полноты. Например, запрос о жизни и достижениях какого-либо ученого может требовать дополнительного поиска по разным аспектам: его открытиям, влиянию, значимости для науки и т.д.


#### 2.3.4. Пример реализации Self-RAG

В LangChain Self-RAG можно реализовать с помощью каскадных запросов, где модель анализирует начальный ответ и формирует новые уточняющие запросы при необходимости. Ниже приведен пример кода, демонстрирующий реализацию Self-RAG.  
Пример основан на LangChain и LangGraph, где модель после начального ответа анализирует его на полноту и, если чего-то не хватает, самостоятельно формирует новый запрос.



In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.retrievers import TFIDFRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Инициализируем базу документов
documents = [
    "Никола Тесла был известен своими разработками в области электротехники и радиосвязи.",
    "Тесла изобрел генератор переменного тока и индукционный мотор.",
    "Работы Теслы стали важной основой для развития современной электротехники.",
    "Его вклад включает эксперименты с беспроводной передачей энергии.",
    "Тесла также изучал влияние электромагнитных волн и создал первую радиостанцию."
]

# Разделяем документы для улучшения поиска
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# Применяем split_text для каждого документа по отдельности
texts = []
for doc in documents:
    texts.extend(text_splitter.split_text(doc))  # Используем split_text для каждого документа

# Создаем список документов для TFIDFRetriever
docs = [Document(page_content=text) for text in texts]
retriever = TFIDFRetriever.from_documents(docs)

# Настройка LLM
llm = OpenAI(temperature=0.2)

# Создаем цепочку для QA с использованием retriever
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

# Определяем запрос
query = "Расскажите о достижениях Никола Тесла в передаче энергии."

# Запуск начального RAG
initial_answer = qa_chain.run(query)
print("Первоначальный ответ:", initial_answer)

# Функция Self-RAG для самостоятельного уточнения ответа
def self_rag(answer, query):
    # Проверяем, упомянуты ли ключевые достижения Теслы
    required_entities = ["передача энергии", "радиосвязь", "электромагнитные волны"]
    missing_entities = [entity for entity in required_entities if entity not in answer]

    if missing_entities:
        # Формируем новый уточняющий запрос
        refined_query = f"{query}. Пожалуйста, укажите: {', '.join(missing_entities)}."
        additional_info = qa_chain.run(refined_query)
        # Объединяем начальный ответ и дополнительную информацию
        full_answer = f"{answer} Дополнительные сведения: {additional_info}"
        return full_answer
    return answer

# Применяем Self-RAG для уточнения ответа
final_answer = self_rag(initial_answer, query)
print("Итоговый ответ с Self-RAG:", final_answer)


Первоначальный ответ:  Никола Тесла был известен своими разработками в области электротехники и радиосвязи, включая эксперименты с беспроводной передачей энергии. Он изобрел генератор переменного тока и индукционный мотор, которые стали основой для современных систем электропередачи. Тесла также изучал влияние электромагнитных волн и создал первую радиостанцию, что позволило ему доказать возможность беспроводной передачи энергии на большие расстояния. Его исследования и изобретения в области передачи энергии сыграли важную роль в развитии современных технологий и оказали значительное влияние на
Итоговый ответ с Self-RAG:  Никола Тесла был известен своими разработками в области электротехники и радиосвязи, включая эксперименты с беспроводной передачей энергии. Он изобрел генератор переменного тока и индукционный мотор, которые стали основой для современных систем электропередачи. Тесла также изучал влияние электромагнитных волн и создал первую радиостанцию, что позволило ему доказать во

**Пояснение**:  

1. **Инициализация базы документов**:
   - создаётся набор текстов, охватывающих основные достижения Никола Теслы.
2. **RAG-процесс**:
   - изначальный запрос находит информацию о достижениях Теслы. Начальный ответ выводится для анализа.
3. **Self-RAG проверка**:
   - определяется, содержатся ли ключевые аспекты (такие как передача энергии или электромагнитные волны) в ответе. Если они отсутствуют, генерируется уточняющий запрос.
4. **Объединение результатов**:
   - если требуется дополнительная информация, она запрашивается и объединяется с начальным ответом, что обеспечивает полноту.

**Пример работы кода**:  

**Первичный запрос**: "Расскажите о достижениях Никола Тесла в передаче энергии."  
**Первоначальный ответ**: "Никола Тесла был известен своими разработками в области электротехники."  
**Уточняющий запрос**: "Расскажите о достижениях Никола Тесла в передаче энергии. Пожалуйста, укажите: радиосвязь, электромагнитные волны."  
**Корректированный ответ**: "Никола Тесла был известен своими разработками в области электротехники. Дополнительные сведения: Тесла также изучал влияние электромагнитных волн и создал первую радиостанцию."  

#### Заключение

Self-RAG представляет собой метод, который позволяет модели работать автономно, контролируя процесс поиска и отбора данных для генерации ответа. Этот подход полезен в ситуациях, где:

- **Требуется автономность:** Self-RAG дает модели возможность выполнять поиск и выбор информации самостоятельно, минимизируя необходимость внешнего управления и обеспечивая гибкость в обработке запросов.
- **Необходимость обработки сложных запросов:** Self-RAG помогает модели разделять и анализировать запросы, адаптируясь к их сложности и при необходимости самостоятельно корректируя выбор ретривера.
- **Требования к адаптивности:** Метод Self-RAG позволяет модели анализировать свой собственный ответ и, при необходимости, корректировать его с помощью дополнительного поиска или уточнения информации.

Self-RAG подходит для интеллектуальных систем, которым требуется максимальная автономность и точность в ответах, например, для использования в научных исследованиях, аналитике и справочных сервисах. Такой подход позволяет создавать ответы на основе широкого спектра данных и улучшает качество взаимодействия за счет самостоятельной коррекции.

### 2.4. Adaptive RAG

#### 2.4.1. Введение в Adaptive RAG

**Adaptive RAG (Adaptive Retrieval-Augmented Generation)** — это метод, при котором модель динамически подбирает наиболее подходящий механизм извлечения информации в зависимости от запроса и его сложности. В отличие от стандартных подходов RAG, которые используют один и тот же тип извлечения для всех запросов, Adaptive RAG анализирует запросы и адаптирует стратегию извлечения, что позволяет повысить точность и релевантность ответов.

**Основные особенности Adaptive RAG:**
- **Гибкость в обработке запросов:** модель может использовать разные подходы к извлечению данных в зависимости от специфики запроса.
- **Оптимизация производительности:** адаптация позволяет использовать простые методы для базовых запросов и более сложные методы для многослойных вопросов, снижая нагрузку на систему.
- **Многоуровневое извлечение:** модель может комбинировать несколько методов извлечения, чтобы сформировать более информативные ответы на сложные запросы.


#### 2.4.2. Проблемы, решаемые Adaptive RAG

1. **Различные уровни сложности запросов:**
   - запросы могут варьироваться от простых фактических вопросов до сложных вопросов, требующих многоступенчатого анализа. Adaptive RAG позволяет модели выбирать подходящий уровень извлечения для каждого типа запроса.
2. **Избежание избыточного извлечения:**
   - стандартные методы RAG могут возвращать больше данных, чем нужно, даже для простых запросов. Adaptive RAG решает эту проблему, снижая объем ненужной информации.
3. **Адаптация к типу запроса:**
   - adaptive RAG позволяет модели анализировать запрос и выбирать, какой тип информации наиболее важен для ответа (например, обобщенные данные или конкретные факты).


#### 2.4.3. Применение Adaptive RAG на практике

Adaptive RAG особенно полезен, когда модель обслуживает различные типы пользователей (например, экспертов и начинающих) или обрабатывает многослойные запросы. Примером могут быть научные или аналитические системы, где модель должна корректно оценить запрос и выбрать подходящую стратегию поиска.


#### 2.4.5. Реализация

Adaptive RAG может быть реализован через каскадный метод выбора ретривера или же с использованием логики, анализирующей длину и сложность запроса. В примере ниже мы рассмотрим реализацию, где модель сначала проверяет сложность запроса и выбирает подходящий механизм извлечения: простое TF-IDF для базовых запросов и векторный поиск на основе косинусной близости для более сложных запросов.

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.retrievers import TFIDFRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chains.question_answering.chain import load_qa_chain
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Документы для примера
documents = [
    "Никола Тесла был известен своими разработками в области электротехники и радиосвязи.",
    "Он изобрел генератор переменного тока и индукционный мотор.",
    "Работы Теслы стали основой для развития современной электротехники.",
    "Тесла изучал беспроводную передачу энергии и электромагнитные волны.",
    "Никола Тесла является одним из самых значимых ученых в истории науки."
]

# Разделяем документы для улучшения поиска
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# Применяем split_text для каждого документа по отдельности
texts = []
for doc in documents:
    texts.extend(text_splitter.split_text(doc))

# Создаем список документов для TFIDFRetriever
docs = [Document(page_content=text) for text in texts]
tfidf_retriever = TFIDFRetriever.from_documents(docs)

# Настраиваем FAISS векторное хранилище
embeddings = OpenAIEmbeddings()
faiss_store = FAISS.from_texts(texts, embeddings)
faiss_retriever = faiss_store.as_retriever()

# Инициализируем модель
llm = OpenAI(temperature=0.2)

# Создаем цепочку вопросов и ответов
llm_chain = load_qa_chain(llm, chain_type="stuff")

# Создаем StuffDocumentsChain с правильным использованием llm_chain
#combine_documents_chain = StuffDocumentsChain(llm_chain=llm_chain)

# Функция для анализа сложности запроса
def is_complex_query(query):
    return len(query.split()) > 5 or any(word in query.lower() for word in ["подробно", "объясните", "детали"])

# Функция Adaptive RAG
def adaptive_rag(query):
    is_complex = is_complex_query(query)
    if is_complex:
        print("Используется RAG с векторным индексом для сложного запроса.")
        retriever = faiss_retriever
    else:
        print("Используется TF-IDF для простого запроса.")
        retriever = tfidf_retriever

    # Создание RAG цепочки с использованием llm и retriever
    rag_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)
    answer = rag_chain.run(query)
    return answer

# Примеры запросов
simple_query = "Кто такой Никола Тесла?"
complex_query = "Объясните достижения Никола Теслы в области передачи энергии и электромагнитных волн подробно."

# Запуск Adaptive RAG для каждого запроса
print("Ответ для простого запроса:", adaptive_rag(simple_query))
print("Ответ для сложного запроса:", adaptive_rag(complex_query))


Используется TF-IDF для простого запроса.
Ответ для простого запроса:  Никола Тесла - известный ученый, который внес значительный вклад в области электротехники и радиосвязи. Он также изучал беспроводную передачу энергии и электромагнитные волны, и его работы стали основой для развития современной электротехники. Он также считается одним из самых значимых ученых в истории науки.
Используется RAG с векторным индексом для сложного запроса.
Ответ для сложного запроса:  Никола Тесла был известен своими разработками в области электротехники и радиосвязи, включая беспроводную передачу энергии и электромагнитные волны. Он провел множество экспериментов и изобрел устройства, которые позволяли передавать энергию без проводов, используя электромагнитные волны. Он также создал первую радио передающую станцию и разработал систему для беспроводной связи на большие расстояния. Работы Теслы стали основой для развития современной электротехники и радиосвязи, и его вклад в науку и технологии до сих пор

**Пояснение к коду**

1. **Инициализация базы документов:** Набор текстов о Тесле позволяет создать базу данных для поисковых запросов, предоставляя информацию для обработки запроса на этапе извлечения.
2. **TF-IDF и FAISS ретриверы:** Настроены два ретривера:
   - `TFIDFRetriever` — для обработки простых запросов.
   - `FAISSRetriever` — для работы со сложными запросами, когда необходим более глубокий анализ.
3. **Функция `is_complex_query`:** Функция проверяет, является ли запрос сложным, анализируя длину запроса или наличие ключевых слов (например, "подробно" или "объясните"), которые предполагают более детальный ответ.
4. **Adaptive RAG логика:** В зависимости от результата `is_complex_query`, выбирается соответствующий ретривер. Если запрос признан сложным, применяется FAISS, в противном случае — TF-IDF.
5. **Запросы и ответы:** На простые запросы система отвечает с помощью TF-IDF, а на сложные — с помощью векторного поиска (FAISS), что оптимизирует процесс поиска и повышает релевантность ответов.

**Пример работы кода**

1. **Простой запрос:** `"Кто такой Никола Тесла?"`
   - **Выбранный ретривер:** TF-IDF
   - **Ответ:** `"Никола Тесла был известен своими разработками в области электротехники и радиосвязи."`
   
2. **Сложный запрос:** `"Объясните достижения Никола Теслы в области передачи энергии и электромагнитных волн подробно."`
   - **Выбранный ретривер:** FAISS
   - **Ответ:** `"Тесла изучал беспроводную передачу энергии и электромагнитные волны. Его эксперименты в этой области стали важной основой для развития науки."`

#### Заключение

Adaptive RAG позволяет модели динамически адаптироваться к типу запроса, что повышает качество и релевантность ответов, а также снижает нагрузку на систему. Это особенно полезно в ситуациях, когда:
- **Запросы различаются по сложности:** Adaptive RAG позволяет подбирать стратегию поиска для каждого запроса индивидуально.
- **Производительность критична:** Использование более простых методов для простых запросов и мощных методов для сложных снижает общую нагрузку.
- **Требуется высокая точность:** Adaptive RAG позволяет более точно отвечать на сложные вопросы, адаптируя глубину ответа под запрос.

Adaptive RAG идеально подходит для профессиональных, научных и аналитических приложений, требующих высокого уровня точности и гибкости при обработке запросов.

### 2.5. Сравнение различных подходов RAG

1. **Simple RAG**:
   - Используется стандартный подход с RetrievalQA и векторным хранилищем.
2. **Corrective RAG**:
   - Введен специальный шаблон для исправления ошибок в ответах.
3. **Self-RAG**:
   - Используется самокоррекция в ответах, дополненная уточняющими деталями.
4. **Adaptive RAG**:
   - Применяется подход с анализом сложности запроса (с использованием TF-IDF для простых запросов и FAISS для сложных).

In [ ]:
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TFIDFRetriever
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering.chain import load_qa_chain

# Подготовка LLM и эмбеддингов
llm = OpenAI(temperature=0.2, max_tokens=512)
embeddings = OpenAIEmbeddings()

# Примеры документов о Николе Тесле
documents = [
    "Никола Тесла был известным изобретателем, который работал в области электричества и магнетизма. Он изобрел трансформатор Теслы, который позволил передавать энергию на большие расстояния.",
    "В 1891 году Тесла продемонстрировал передачу энергии без проводов, что стало важным шагом в развитии беспроводных технологий.",
    "Исследования Теслы в области высокочастотного переменного тока привели к разработке систем, способных передавать электроэнергию без использования проводов на значительные расстояния.",
    "Работы Теслы легли в основу многих современных технологий, включая радиосвязь и системы передачи данных.",
    "Тесла также экспериментировал с беспроводной передачей информации и энергии, создав прототипы, которые значительно опережали время. Его работы по радиоволнам стали основой для дальнейших достижений в области телекоммуникаций.",
    "В 1899 году Тесла провел знаменитые эксперименты в Колорадо-Спрингс, где ему удалось создать искусственные молнии, что продемонстрировало огромный потенциал высокочастотных электрических волн.",
    "Тесла был одним из пионеров в области разработки технологий для создания новых типов электродвигателей и генераторов, которые стали основой для последующих инноваций в промышленности.",
    "В 1917 году Тесла предложил концепцию мирового беспроводного радио и связи, которая предвосхитила современную мобильную коммуникацию.",
    "Хотя многие из его идей были недооценены в свое время, Тесла оставил неизгладимый след в истории науки и технологий, особенно в области энергетики и телекоммуникаций."
]

# Разделение текста на куски для обеспечения правильного поиска
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# Применяем split_text для каждого документа по отдельности
texts = []
for doc in documents:
    texts.extend(splitter.split_text(doc))

# Создаем список документов для TFIDFRetriever
docs = [Document(page_content=text) for text in texts]



# Подготовка векторного хранилища для VectorstoreRetriever
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore_retriever = vectorstore.as_retriever()

# TF-IDF retriever
tfidf_retriever = TFIDFRetriever.from_texts([doc.page_content for doc in docs])

# Шаблон для Corrective RAG
corrective_template = """Ответьте на запрос, используя следующие документы:
{context}
Запрос: {query}
Коррекция: если в ответе есть ошибка или неверная информация, исправьте ее.

Ответ:"""
corrective_prompt = PromptTemplate(input_variables=["context", "query"], template=corrective_template)

# Шаблон для Self-RAG
self_rag_template = """Ответьте на запрос, используя следующие документы:
{context}
Запрос: {query}
Используйте самокоррекцию, чтобы избежать ошибок в ответе и добавьте уточняющие детали.

Ответ:"""
self_rag_prompt = PromptTemplate(input_variables=["context", "query"], template=self_rag_template)

# Подготовка цепочек для каждого подхода
simple_rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore_retriever)
corrective_rag = LLMChain(llm=llm, prompt=corrective_prompt)
self_rag = LLMChain(llm=llm, prompt=self_rag_prompt)

# Настройка Adaptive RAG
# Инициализация FAISS и настройки цепочек
faiss_store = FAISS.from_texts([doc.page_content for doc in docs], embeddings)
faiss_retriever = faiss_store.as_retriever()

# Функция для анализа сложности запроса
def is_complex_query(query):
    return len(query.split()) > 5 or any(word in query.lower() for word in ["подробно", "объясните", "детали"])

# Функция Adaptive RAG
def adaptive_rag(query):
    is_complex = is_complex_query(query)
    if is_complex:
        print("Используется RAG с векторным индексом для сложного запроса.\n\n")
        retriever = faiss_retriever
    else:
        print("Используется TF-IDF для простого запроса.\n\n")
        retriever = tfidf_retriever

    # Создание RAG цепочки с использованием llm и retriever
    rag_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)
    answer = rag_chain.run(query)
    return answer

# Функция для выполнения запроса с каждым подходом
def compare_rag_approaches(query):
    responses = {}

    # Simple RAG
    responses['Simple RAG'] = simple_rag.run(query)

    # Corrective RAG
    context = vectorstore_retriever.get_relevant_documents(query)
    responses['Corrective RAG'] = corrective_rag.run({"context": context, "query": query})

    # Self-RAG
    responses['Self-RAG'] = self_rag.run({"context": context, "query": query})

    # Adaptive RAG
    responses['Adaptive RAG'] = adaptive_rag(query)

    return responses

# Пример использования
#query = "Объясните достижения Никола Теслы в области передачи энергии и электромагнитных волн подробно."
query = "Расскажи подробно про Никола Тесла."
results = compare_rag_approaches(query)

# Вывод результатов для сравнения
for approach, response in results.items():
    print(f"{approach} ответ:\n{response}\n{'-'*50}\n")


Используется RAG с векторным индексом для сложного запроса.
Simple RAG ответ:
 Никола Тесла был известным изобретателем, который работал в области электричества и магнетизма. Он изобрел трансформатор Теслы, который позволил передавать энергию на большие расстояния. Тесла был одним из пионеров в области разработки технологий для создания новых типов электродвигателей и генераторов, которые стали основой для последующих инноваций в промышленности. Он также провел знаменитые эксперименты в Колорадо-Спрингс, где ему удалось создать искусственные молнии, что продемонстрировало огромный потенциал высокочастотных электрических волн. Хотя многие из его идей были недооценены в свое время, Тесла оставил неизгладимый след в истории науки и технологий, особенно в области энергетики и телекоммуникаций. В целом, Никола Тесла был великим ученым и изобретателем, чьи достижения продолжают влиять на нашу жизнь и сегодня.
--------------------------------------------------

Corrective RAG ответ:
 Никола Т